In [103]:
import random, math

In [104]:
def geraIndividuo(tamCromossomo):
    
    cromossomo = []
    
    while tamCromossomo > 0:
        cromossomo.append(random.randint(0,1))
        tamCromossomo -= 1
        
    return cromossomo

In [105]:
def geraPopulacao(tamPop, tamCromossomo):
    
    populacao = []
    
    while tamPop > 0:
        populacao.append(geraIndividuo(tamCromossomo))
        tamPop -= 1
        
    return populacao

In [106]:
#argumentos: recebe array binario do cromossomo (geraPopulacao[i])
#retorna: o valor real x e y do cromossomo

def cromossomoReal(cromossomo):
    
    x = 0
    y = 0
    i = 0
    
    while(i < len(cromossomo)/2):
        x += cromossomo[i] * pow(2, len(cromossomo)/2-i-1)
        i += 1
        
    while(i < len(cromossomo)):
        y += cromossomo[i] * pow(2, len(cromossomo)-i-1)
        i += 1

    return [x,y]

In [107]:
#argumentos: recebe um valor real e o tamanho do cromossomo (len(geraPopulacao[i]))
#retorna: o valor real x e y do cromossomo dentro do intervalo da funcao

def valorNoIntervalo(a, cromossomoLength):
    
    a = -5.12 + (10.24 / (pow(2,cromossomoLength / 2) - 1) * a)
    
    return a

In [108]:
#argumentos: recebe um cromossomo
#retorna: o valor real fitness do individuo

def fitness(cromossomo):
    
    valorReal = cromossomoReal(cromossomo)
    x = valorNoIntervalo(valorReal[0], len(cromossomo))
    y = valorNoIntervalo(valorReal[1], len(cromossomo))
    cosx = math.cos(2 * math.pi * x)
    cosy = math.cos(2 * math.pi * y)
    
    resultado = 1/(20 + pow(x,2) + pow(y,2) - 10*(cosx+cosy))
    
    return resultado

In [124]:
#argumentos: a populacao toda
#retorna: o cromossomo de um individuo

def selecao(pop):
    
    fitnessAcumulado = []
    fitnessTotal = 0
    
    for i in pop:
        fitnessAtual = fitness(i)
        fitnessTotal += fitnessAtual
        fitnessAcumulado.append(fitnessTotal)
    
    selecionar = random.random()*fitnessTotal
    
    j = 0
    while(j < len(pop)):
        if(fitnessAcumulado[j] >= selecionar):
            return pop[j]
        else:
            j += 1

In [110]:
#argumentos: dois cromossomos (selecao(pop))
#retorno: um array contendo dois cromossomos

def crossover(pai, mae):
    
    ponto = random.randint(1,len(pai)-1)
    i = 0
    filhos = []
    filhoUm = []
    filhoDois = []
    
    while(i < len(pai)):
        if(i < ponto):
            filhoUm.append(pai[i])
            filhoDois.append(mae[i])
        else:
            filhoUm.append(mae[i])
            filhoDois.append(pai[i])
        i += 1
        
    filhos.append(filhoUm)
    filhos.append(filhoDois)
    
    return filhos

In [111]:
def mutacao(cromossomo):
    
    r = random.randint(0,len(cromossomo)-1)
    
    if(cromossomo[r] == 1):
        cromossomo[r] = 0
    else:
        cromossomo[r] = 1
        
    return cromossomo

In [112]:
def novaGeracao(pop, chanceMutacao):
    
    novaPop = []
    
    while(len(novaPop) < len(pop)):
        r = random.random()
        
        if(r <= chanceMutacao):
            novaPop.append(mutacao(selecao(pop)))
            
        else:
            filhos = crossover(selecao(pop), selecao(pop))
            novaPop.append(filhos[0])
            if(len(novaPop) < len(pop)):
                novaPop.append(filhos[1])
        
    return novaPop

In [117]:
def fitnessGeracao(pop):
    
    fitnessGeracao = 0
    i = 0
    
    for i in pop:
        fitnessGeracao += fitness(i)
    
    return fitnessGeracao/len(pop)

In [126]:
def maiorFitness(pop):
    
    maior = 0
    
    for i in pop:
        atual = fitness(pop[i])
        
        if(maior < atual):
            maior = atual
    
    return maior

In [132]:
def rodaGeracoes(numGen, tamPop, tamCromossomo, chanceMutacao, verbose):
    
    pop = geraPopulacao(tamPop, tamCromossomo)
    i = 0
    maior = 0
    maiorCromossomo = []
    maiorGen = 0
    
    while(i < numGen):
        j = 0
        while(j < len(pop)):
            fitAtual = fitness(pop[j])
            if(fitAtual > maior):
                if(verbose == True): print("Novo maior fitness (gen", i, ')', fitAtual, 'Antigo:', maior)
                maior = fitAtual
                maiorGen = i
                maiorCromossomo = pop[j]
            j += 1
            
        if(verbose == True): print('Gen', i, 'Melhor fitness:', maior, 'Fitness medio:', fitnessGeracao(pop))
        pop = novaGeracao(pop, chanceMutacao)
        i += 1

In [134]:
rodaGeracoes(40, 100, 20, 0.1, True)

Novo maior fitness (gen 0 ) 0.021740252782971564 Antigo: 0
Novo maior fitness (gen 0 ) 0.0310896532145737 Antigo: 0.021740252782971564
Novo maior fitness (gen 0 ) 0.034407925832730536 Antigo: 0.0310896532145737
Novo maior fitness (gen 0 ) 0.034474625368286285 Antigo: 0.034407925832730536
Novo maior fitness (gen 0 ) 0.10082906097737213 Antigo: 0.034474625368286285
Novo maior fitness (gen 0 ) 0.12323439860327852 Antigo: 0.10082906097737213
Gen 0 Melhor fitness: 0.12323439860327852 Fitness medio: 0.03178904060646755
Novo maior fitness (gen 1 ) 0.48500605032843047 Antigo: 0.12323439860327852
Gen 1 Melhor fitness: 0.48500605032843047 Fitness medio: 0.046724856476514594
Novo maior fitness (gen 2 ) 0.508575248981534 Antigo: 0.48500605032843047
Gen 2 Melhor fitness: 0.508575248981534 Fitness medio: 0.06444155113308009
Novo maior fitness (gen 3 ) 0.6617457286012561 Antigo: 0.508575248981534
Gen 3 Melhor fitness: 0.6617457286012561 Fitness medio: 0.11376107988471101
Novo maior fitness (gen 4 ) 0

In [ ]:
cr = (0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1)
valorReal = cromossomoReal(cr)
x = valorNoIntervalo(valorReal[0], len(cr))
y = valorNoIntervalo(valorReal[1], len(cr))
print(valorReal, x, y)
print(fitness(cr))